<a href="https://colab.research.google.com/github/aleks-haksly/KarpovCources_Hard_DA_advanced/blob/main/03%20-%20Advanced%20experiments%20design/02_mde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импортируем используемые в работе модули
import pandas as pd
!pip install clickhouse-driver --q
from clickhouse_driver import Client
from google.colab import userdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.7/935.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
# Реквизиты для подключения к БД
user="hardda_student_alekse-fomin"
password=userdata.get('HA[hardda_student_alekse-fomin]')

host='clickhouse.lab.karpov.courses'
port=9000
database_r='hardda'

In [ ]:
# Создаем соединение с БД ClickHouse
client_r = Client(host=host, port=port, user=user,
                password=password, database=database_r)

### Задание 1 ###
Подготовьте SQL запрос для клиентов, совершивших оплату в Москве в период с 11 по 16 октября 2022 г. за услугу "premium".

In [ ]:
sql = """WITH transactions as (
  SELECT
    payment_date,
    advert_id,
    amount,
    passport_id
  FROM
    user_transactions t
  WHERE
    t.payment_date BETWEEN '2022-10-11'
    and '2022-10-16'
    AND t.sign = -1
    and t.type = 'premium'
    and t.amount != 0
)

SELECT
  t.passport_id,
  abs(sum(t.amount)) AS revenue
FROM
  transactions AS t
LEFT JOIN
  dp_live_adverts AS a ON CAST(a.advert_id, 'UInt32') = t.advert_id AND a.execution_date = t.payment_date
WHERE
  region = 'Москва'
GROUP BY
  t.passport_id
"""
result, columns = client_r.execute(sql, with_column_types=True)
df = pd.DataFrame(result, columns=[tuple[0] for tuple in columns]) #.set_index('payment_date')
#df['payment_date'] = pd.to_datetime(df['payment_date'], format="yyyy-mm-dd")
#df['tstamp'] = pd.to_datetime(df['tstamp'], format="yyyy-mm-dd hh:mm:ss")

In [ ]:
df

,passport_id,revenue
0,140977147,985
1,134882393,1105
2,139901702,1100
3,123673877,1100
4,142445756,1100
...,...,...
1048,124592490,1105
1049,135976735,1100
1050,123466729,1100
1051,142277608,1105


### Задание 2 ###
В продукте планируется внедрение нового функционала с целью улучшить привлекательность услуги "premium". Менеджеры предполагают, что это позволит увеличить среднюю выручку на 8% (поле amount).

Вам необходимо определить минимально необходимый размер выборки, чтобы статистически значимо задетектить эффект при увеличении выручки на 8% с учетом мощности теста в 80% и установленного уровня значимости alpha = 0.05.

Давайте решать эту задачу по шагам. В первую очередь, нам необходимо определить среднюю выручку и стандартное отклонение. И найти ожидаемую менеджером среднюю выручку (ARPPU) с учетом ее увеличения на 8%.

In [ ]:
ARPPU = round(df.revenue.mean() * 1.08, 2)
print("Ожидаемая менеджером средняя выручка (ARPPU) с учетом ее увеличения на 8%% составляет %s " % ARPPU)

Ожидаемая менеджером средняя выручка (ARPPU) с учетом ее увеличения на 8% составляет 1754.88 


### Задание 3 ###
Найдите значение для стандартизированного эффекта (индекс Коэна).

In [ ]:
cohens_d = (ARPPU - df.revenue.mean()) / df.revenue.std()
print("Значение для стандартизированного эффекта (индекс Коэна) составляет %s " % round(cohens_d, 2))

Значение для стандартизированного эффекта (индекс Коэна) составляет 0.08 


### Задание 4 ###
Используя рассчитанные значения, найдите минимально необходимый размер выборки, чтобы статистически значимо задетектить эффект.
Рассчитываем минимальный размер выборки, необходимый, чтобы стат.значимо задетектить разницу с вероятностью совершить ошибку II рода ниже 20% при уровне значимости альфа = 0.05:

In [ ]:
from statsmodels.stats.power import tt_ind_solve_power

In [ ]:
sample_size = round(tt_ind_solve_power(effect_size=cohens_d,
                                       nobs1=None, # Его мы и ищем
                                       alpha=0.05,
                                       power=0.8,
                                       ratio=1.0,
                                       alternative='two-sided'))
print('Размер требуемой выборки для каждой группы не менее: %s' % sample_size)

Размер требуемой выборки для каждой группы не менее: 2369


### Задание 5 ###
Размер выборки с новыми параметрами эксперимента.
Менеджер предложил снизить ошибку II рода до 10%, при этом относительный эффект, который требуется задетектить, был установлен в 6%. Уровень значимости alpha остался прежним (0.05).
Оцените размер выборки, который вам потребуется, чтобы задетектить стат.значимый эффект от нововведения, если он присутствует.

In [ ]:
ARPPU_new = (df.revenue.mean() * 1.06)
cohens_d_new = (ARPPU_new - df.revenue.mean()) / df.revenue.std()
sample_size = round(tt_ind_solve_power(effect_size=cohens_d_new,
                                       nobs1=None, # Его мы и ищем
                                       alpha=0.05,
                                       power=0.9,
                                       ratio=1.0,
                                       alternative='two-sided'))
print('Размер требуемой выборки для каждой группы не менее: %s' % sample_size)

Размер требуемой выборки для каждой группы не менее: 5636


### Задание 6 ###
Имеется несбалансированная выборка из 100 тыс. пользователей, где соотношение между группами составляет 80:20 (на 20% пользователей распространяется нововведение, а на 80% нет: ratio = n2/n1 = 0.25). Дизайн теста предполагает вероятность в 5% совершить ошибку I рода и допущение в 20% совершить ошибку II рода.

Определите размер эффекта, который мы сможем задетектить при заданном дизайне эксперимента.

In [ ]:
MDE = round(tt_ind_solve_power(effect_size=None, # Его мы и ищем
                                       nobs1=80_000,
                                       alpha=0.05,
                                       power=0.8,
                                       ratio=0.25,
                                       alternative='two-sided'), 3)
print('Минимальный размер стандартизированного эффекта (cohen`s d MDE) который можно зафиксировать при заданном дизайне эксперимента: %s' % MDE)

Минимальный размер стандартизированного эффекта (cohen`s d MDE) который можно зафиксировать при заданном дизайне эксперимента: 0.022


### Задание 7 ###
Максимальный размер в двух выборках, доступный вам, равен 100 тыс (суммаа тестовой и контрольной выборок). пользователей. Определите, при каком соотношении между контрольной и тестовой группами (n2/n1) мы сможем задетектить минимальное значение размера эффекта (Cohen's D) при уровне значимости alpha = 0.05 и мощности теста = 80% и стат критерии t-test.

In [ ]:
import numpy as np

max_population = 100000 # размер выборки

# подготовим списки из различного числа пользователей, для различных соотношений
n1 = np.arange(5000, 95000, 5000)
effect_list = []
ratio_list = []
# определим величину эффекта для всех вариантов соотношений
for i in range(len(n1)):
    effect_result = power.tt_ind_solve_power(effect_size = None,
                            power = 0.8,
                            nobs1 = n1[i],
                            ratio = (max_population - n1[i]) / n1[i],
                            alpha = 0.05)
    ratio = (100000 - n1[i]) / n1[i]
    ratio_list.append(ratio)
    effect_list.append(effect_result)
best_ratio = ratio_list[np.argmax(effect_list == np.min(effect_list))]

print('Минимальной ratio n2/n1 = {}'.format(best_ratio))


Минимальной ratio n2/n1 = 1.0


### Задание 8 ###
Чему будет равна величина минимального эффекта - Cohen's D?


In [ ]:
cohen_d = tt_ind_solve_power(effect_size = None,
                            power = 0.8,
                            nobs1 = max_population / 2,
                            ratio = 1,
                            alpha = 0.05)
print('Минимальный размер стандартизированного эффекта (cohen`s d MDE) который можно зафиксировать при заданном дизайне эксперимента: %s' % round(cohen_d, 3))

Минимальный размер стандартизированного эффекта (cohen`s d MDE) который можно зафиксировать при заданном дизайне эксперимента: 0.018
